# Image Classification - The Multi-class Weather Dataset

## Data exploration, preparation, and partition

Please start by downloading the Mendeley Weather Dataset (MWD) from the following link and unzip it:

https://data.mendeley.com/datasets/4drtyfjtfy/1

Once unzipped, you’ll find the dataset contains 1,125 images representing various weather conditions. Ensure these images are placed in a folder named dataset2, located in the same directory as your Jupyter notebook.

Create three CSV files—my_training.csv, my_validation.csv, and my_test.csv—to split the dataset into training, validation, and test sets. Each CSV file should include the following columns:

- File path
- Image label

Ensure that the dataset is divided randomly to maintain a uniform distribution of labels across each set. The entries should be sorted randomly.




# Importing Assets

In [27]:
import csv
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from keras_tuner import RandomSearch, HyperParameters

#### 1.1.2 Extracting file paths and labels

First we prepares an image dataset by iterating through files in dataset2, extracting labels from file names, and storing both file paths and labels in a pandas DataFrame. We construct full file paths, extracts text labels while removing digits and trailing spaces, and populates lists with these paths and labels. Finally, we create a DataFrame with two columns for file paths and labels for further processing of the dataset

In [2]:
# Initializing lists to hold paths and labels
file_paths = []
labels = []

# Loop through each file in dataset2 
for file_name in os.listdir('dataset2'):
    if file_name.endswith('.jpg'):
        
        # Construct the full file path
        file_path = os.path.join('dataset2', file_name)
        
        # Extract the label from the file name by 
        
        label = file_name.split('.')[0]   # removing all after dot
        label = ''.join([i for i in label if not i.isdigit()]).rstrip()  # removing whitespace after lables
        
        file_paths.append(file_path)
        labels.append(label)

# Creating our DataFrame
data = pd.DataFrame({
    'File path': file_paths,
    'Image label': labels
})

#### 1.1.3 Making random partitions with similar distribution

Next we Split the dataset into training, validation, and test sets for effective training, tuning, and evaluation of models.

In [3]:
# Shuffle data
data = data.sample(frac=1).reset_index(drop=True)

# Split the dataset into training (60%), validation (20%), and test sets (20%)
train, temp = train_test_split(data, test_size=0.4, stratify=data['Image label']) # Training and Temporary Set
validation, test = train_test_split(temp, test_size=0.5, stratify=temp['Image label']) # Validation and Test Set

# Save the partitions to their corresponding CSV files
train.to_csv('my_training.csv', index=False, header=False)
validation.to_csv('my_validation.csv', index=False, header=False)
test.to_csv('my_test.csv', index=False, header=False)

#### 1.1.4 Data Partitioning Results

Now we compare our results with the given dataset to ensure consistency

In [4]:
# Load the partitions from CSV files
train_df = pd.read_csv('my_training.csv', header=None, names=['File path', 'Image label'])
validation_df = pd.read_csv('my_validation.csv', header=None, names=['File path', 'Image label'])
test_df = pd.read_csv('my_test.csv', header=None, names=['File path', 'Image label'])

# Creating a function to show label distribution and our first 10 rows
def display_info(df, partition_name):
    print(f"Label Distribution in {partition_name}:")
    print(df['Image label'].value_counts(normalize=True), '\n'*2)  # Normalized count for distribution
    
    # first 10 row display
    print(f"First 10 Rows of {partition_name}:")
    print(df.head(10), '\n'*2)  

# Display the information for each partition
display_info(train_df, 'Training Set')
display_info(validation_df, 'Validation Set')
display_info(test_df, 'Test Set')

Label Distribution in Training Set:
sunrise    0.316493
cloudy     0.267459
shine      0.225854
rain       0.190193
Name: Image label, dtype: float64 


First 10 Rows of Training Set:
                 File path Image label
0      dataset2/rain43.jpg        rain
1   dataset2/cloudy211.jpg      cloudy
2     dataset2/rain152.jpg        rain
3    dataset2/shine224.jpg       shine
4   dataset2/sunrise57.jpg     sunrise
5   dataset2/sunrise85.jpg     sunrise
6  dataset2/sunrise144.jpg     sunrise
7    dataset2/shine109.jpg       shine
8    dataset2/shine232.jpg       shine
9    dataset2/cloudy90.jpg      cloudy 


Label Distribution in Validation Set:
sunrise    0.316964
cloudy     0.267857
shine      0.223214
rain       0.191964
Name: Image label, dtype: float64 


First 10 Rows of Validation Set:
                 File path Image label
0   dataset2/sunrise68.jpg     sunrise
1     dataset2/shine17.jpg       shine
2   dataset2/cloudy212.jpg      cloudy
3    dataset2/cloudy56.jpg      cloudy
4

#### 1.1.5 Comparing Our Results with the Given Dataset

In [5]:
df_test = pd.read_csv('test.csv', header=None, names=['File path', 'Image label'])
df_train = pd.read_csv('training.csv', header=None, names=['File path', 'Image label'])
df_val = pd.read_csv('validation.csv', header=None, names=['File path', 'Image label'])

display_info(df_train, 'Given Training Set')
display_info(df_val , 'Given Validation Set')
display_info(df_test, 'Given Test Set')

Label Distribution in Given Training Set:
sunrise    0.327785
cloudy     0.256082
shine      0.227913
rain       0.188220
Name: Image label, dtype: float64 


First 10 Rows of Given Training Set:
                 File path Image label
0    dataset2/shine137.jpg       shine
1    dataset2/shine177.jpg       shine
2    dataset2/cloudy87.jpg      cloudy
3  dataset2/sunrise290.jpg     sunrise
4     dataset2/shine88.jpg       shine
5  dataset2/sunrise275.jpg     sunrise
6    dataset2/cloudy40.jpg      cloudy
7  dataset2/sunrise313.jpg     sunrise
8    dataset2/cloudy56.jpg      cloudy
9   dataset2/cloudy250.jpg      cloudy 


Label Distribution in Given Validation Set:
sunrise    0.305389
cloudy     0.281437
shine      0.227545
rain       0.185629
Name: Image label, dtype: float64 


First 10 Rows of Given Validation Set:
                 File path Image label
0     dataset2/shine14.jpg       shine
1    dataset2/cloudy47.jpg      cloudy
2     dataset2/rain118.jpg        rain
3  dataset2/sunr

Both datasets and distributions look reasonably similar.**

### 1.2 - preprocessing and preparation

I'll use TensorFlow's TextLineDataset to create datasets for my training, validation, and test phases. My datasets will generate images that are resized to dimensions of 230 x 230 with 3 channels. Then normalized to fall within the range of 0 to 1.

#### 1.2.1 Encoding our data 

We define and use `encode_labels` function to read a CSV file containing image paths and their corresponding textual labels, replace these textual labels with numeric codes based on a predefined dictionary `label_to_index`, and save the results to a new CSV file.

In [6]:
label_to_index = {'cloudy': 0, 'rain': 1, 'sunrise': 2, 'shine': 3}

#function to encode labels from CSV
def encode_labels(csv_file_path, output_file_path):
    # Open input CSV for reading ('r') and output for writing ('w').
    with open(csv_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        for row in reader:

            # Use the dictionary 'label_to_index' to find the encoded value, -1 if not found            
            writer.writerow([row[0], label_to_index.get(row[1], -1)])


encode_labels('training.csv', 'training_encoded.csv')
encode_labels('validation.csv', 'validation_encoded.csv')
encode_labels('test.csv', 'test_encoded.csv')


Next, we sequence functions for loading and preprocessing our image dataset within a TensorFlow pipeline. The workflow involves reading image paths and labels from CSV files, decoding and processing images, and preparing TensorFlow datasets.

#### 1.2.2 Resizing, normalizing, parsing, and decoding 

In [7]:
import tensorflow as tf

def parse_image_and_label(line):
    parts = tf.strings.split(line, ',')
    image = tf.io.read_file(parts[0])  # Read the image from the path -> first column -> part[0].
    image = tf.image.decode_jpeg(image, channels=3) #Decode into a RGB
    image = tf.image.resize(image, [230, 230]) # Resize into 230x230 
    image = image / 255.0  # Normalize to [0, 1]
    
    # ensures that the label is treated as a numeric type by converting string -> 32-bit int
    label = tf.strings.to_number(parts[1], tf.int32)
    return image, label

def load_dataset(file_path):
    dataset = tf.data.TextLineDataset(file_path)
    #instructs TensorFlow to autodetermine  optimal threads to use for parallelizing the map function. 
    dataset = dataset.map(parse_image_and_label, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

# Creating the datasets
training_dataset = load_dataset('training_encoded.csv')
validation_dataset = load_dataset('validation_encoded.csv')
test_dataset = load_dataset('test_encoded.csv')

Now, `training_dataset`, `validation_dataset`, and `test_dataset` are ready to be used for model training and evaluation.


## A simple classifier

### 2.1 First classifier

I'll start by designing a basic model that includes:
- A Flatten layer 
- An output layer configured with the appropriate size and activation function for our classification task. 

After setting up the model, I'll proceed to train it using the training data. hroughout the training process, I'll utilize the validation data to decide when it's optimal to stop training, ensuring the model doesn't overfit. Once the training is complete, I'll evaluate the trained model's performance on the test data and share the accuracy achieved.

#### 2.1.1 Simple Model Compiling

This simple neural network serves as a baseline model to evaluate performance before moving on to more complex architectures.

In [9]:
model_simple = tf.keras.Sequential([
    # Adding layers
    tf.keras.layers.Flatten(input_shape=(230, 230, 3)),
    tf.keras.layers.Dense(4, activation='softmax') # predicts one of four classes
])


model_simple.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy', #labels are integers -> sparse_categorical_crossentropy
              metrics=['accuracy'])


early_stopping_simple = EarlyStopping(
    monitor='val_loss', # Monitor validation loss
    patience=3,         # Number of epochs with no improvement after which training will be stopped
    verbose = 1,        # Show logs
    restore_best_weights=True # Restores model weights from the epoch with the best value of the monitored metric
)

#### 2.1.2 Training the model

In [10]:
simple_history = model_simple.fit(training_dataset,
                    epochs=100,  # early stopping will prevent it from reaching this
                    validation_data=validation_dataset,
                    callbacks=[early_stopping_simple])

Epoch 1/100
      1/Unknown - 0s 120ms/step - loss: 1.4758 - accuracy: 0.1562

2024-04-05 18:15:50.466675: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2024-04-05 18:15:50.478947: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


     22/Unknown - 0s 15ms/step - loss: 59.4619 - accuracy: 0.4205

2024-04-05 18:15:51.158557: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


25/25 [==============================] - 1s 32ms/step - loss: 57.1350 - accuracy: 0.4264 - val_loss: 10.9868 - val_accuracy: 0.6467
Epoch 2/100
25/25 [==============================] - 1s 24ms/step - loss: 23.4487 - accuracy: 0.5980 - val_loss: 13.7642 - val_accuracy: 0.6168
Epoch 3/100
25/25 [==============================] - 1s 22ms/step - loss: 24.4147 - accuracy: 0.5762 - val_loss: 16.3793 - val_accuracy: 0.5868
Epoch 4/100
25/25 [==============================] - 1s 23ms/step - loss: 22.1682 - accuracy: 0.6287 - val_loss: 13.4713 - val_accuracy: 0.6228
Epoch 4: early stopping


#### 2.1.3 Simple Model Results

In [11]:
# Evaluating the model on the test dataset
test_loss_simple, test_acc_simple = model_simple.evaluate(test_dataset)

print(f"Test Loss: {test_loss_simple}")
print(f'\nTest accuracy: {test_acc_simple*100:.2f}%')

6/6 [==============================] - 0s 13ms/step - loss: 12.2706 - accuracy: 0.6154
Test Loss: 12.27064323425293

Test accuracy: 61.54%


2024-04-05 18:15:54.813915: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


### 2.2 A more complex classifier
I'll experiment with a more intricate architecture, incorporating one or more hidden layers along with dropout regularization. To streamline this process and optimize the architecture, I'll employ keras-tuner. I'll explore various parameters, including the number and sizes of hidden layers, dropout rates, and learning rates. By tuning these parameters, I aim to find the configuration that yields the best performance for our task.

#### 2.2.1 Complex Model Compiling

Next, we set up a more complex and configurable neural network architecture for hyperparameter tuning using TensorFlow's Keras API and the Keras Tuner. It dynamically adjusts the model's architecture and parameters based on the performance on a validation dataset. 

In [12]:
def hypermodel(hp):
    model = tf.keras.Sequential([
        Flatten(input_shape=(230, 230, 3))
    ])
    
    for i in range(hp.Int('num_hidden_layers', 1, 4)):
        model.add(Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=256, step=32),
            activation='relu'
        ))
        model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)))
    
    model.add(Dense(4, activation='softmax'))

    optimizer_choice = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd'])
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    
    # Using legacy optimizers for compatibility with M1/M2 Macs
    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.legacy.RMSprop(learning_rate=learning_rate)
    elif optimizer_choice == 'sgd':
        optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model



tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=10, 
    executions_per_trial=2,  # Performing 2 executions per trial
    directory='my_dir',
    project_name='complex_model_tuning'
)

early_stopping_complex = EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=1,
    restore_best_weights=True
)

tuner.search(training_dataset,
             epochs=100, 
             validation_data=validation_dataset,
             callbacks=[early_stopping_complex],
             verbose=1
)


Trial 10 Complete [00h 00m 46s]
val_accuracy: 0.78742516040802

Best val_accuracy So Far: 0.814371258020401
Total elapsed time: 00h 04m 52s


#### 2.2.2 Retrieving the best set of hyperparameters

In [13]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model_complex_final = tuner.hypermodel.build(best_hps)

# Details of the best hyperparameters
num_hidden_layers = best_hps.get('num_hidden_layers')
print(f"Number of hidden layers: {num_hidden_layers}")

for i in range(num_hidden_layers):
    print(f"Hidden layer {i+1} size: {best_hps.get(f'units_{i}')}")
    print(f"Dropout rate for layer {i+1}: {best_hps.get(f'dropout_{i}')}")
    
optimizer_choice = best_hps.get('optimizer')
learning_rate = best_hps.get('learning_rate')
print(f"Optimizer: {optimizer_choice}")
print(f"Learning rate: {learning_rate}")

Number of hidden layers: 1
Hidden layer 1 size: 256
Dropout rate for layer 1: 0.1
Optimizer: sgd
Learning rate: 0.00483404618006194


#### 2.2.3 Building & Training the Best Fit Model

In [14]:
# Training the complex model
complex_history = model_complex_final.fit(
    training_dataset,
    epochs=100,  # EarlyStopping can stop it earlier
    validation_data=validation_dataset,
    callbacks=[early_stopping_complex]
)


Epoch 1/100
25/25 [==============================] - 2s 57ms/step - loss: 3.4603 - accuracy: 0.4277 - val_loss: 0.9035 - val_accuracy: 0.6168
Epoch 2/100
25/25 [==============================] - 1s 53ms/step - loss: 0.9124 - accuracy: 0.6133 - val_loss: 0.7915 - val_accuracy: 0.7126
Epoch 3/100
25/25 [==============================] - 1s 53ms/step - loss: 0.8054 - accuracy: 0.6543 - val_loss: 0.7420 - val_accuracy: 0.6407
Epoch 4/100
25/25 [==============================] - 1s 54ms/step - loss: 0.7251 - accuracy: 0.6825 - val_loss: 0.7105 - val_accuracy: 0.6826
Epoch 5/100
25/25 [==============================] - 1s 56ms/step - loss: 0.6651 - accuracy: 0.7234 - val_loss: 0.6225 - val_accuracy: 0.7844
Epoch 6/100
25/25 [==============================] - 1s 53ms/step - loss: 0.6239 - accuracy: 0.7388 - val_loss: 0.6059 - val_accuracy: 0.7725
Epoch 7/100
25/25 [==============================] - 1s 53ms/step - loss: 0.6168 - accuracy: 0.7580 - val_loss: 0.5914 - val_accuracy: 0.8024
Epoch 

#### 2.2.4 Test Results of Our Complex Model

In [15]:
# Evaluating the model on the test dataset
test_loss_complex, test_accuracy_complex = model_complex_final.evaluate(test_dataset)

print(f"Test Loss: {test_loss_complex}")
print(f'Test accuracy: {test_accuracy_complex*100:.2f}%')

6/6 [==============================] - 0s 14ms/step - loss: 0.4786 - accuracy: 0.8462
Test Loss: 0.4786064624786377
Test accuracy: 84.62%


Write text below where you explain and justify your decision choices made in this task.

(write your answer here)

#### 2.2.5 Explaining Decision Choices

In the process of designing the second model architecture I used `keras-tuner` to optimize hyperparameters

- **Number of Hidden Layers**:

Allowed the tuner to select between 1 and 4 hidden layers as adding hidden layers to neural networks can  increase  capacity to learn more patterns in the data. However, too many layers can lead to overfitting. A range of 1 to 4 layers is a moderate choice here.

- **Sizes of Hidden Layers**:

For each layer, the number of units ranges from 32 to 256, with a step of 32. This range allows the model to explore various sizes of layers to understand how much capacity (number of neurons) is needed in each layer to learn the patterns in the data effectively.

- **Dropout Rate**:

The dropout rate for each layer ranges from 0.1 to 0.5, with a step of 0.1. This range is chosen to find a sweet spot where the model is regularized enough to generalize well but not too much to underfit by losing significant information.

- **Optimizer:**

The choice among `adam`, `rmsprop`, and `sgd` covers a broad spectrum of optimization algorithms, from adaptive learning rate methods (adam and rmsprop) to a more classical approach (sgd). This allows the tuning process to evaluate both the efficiency and effectiveness of different types of optimizers in the model's training process.
**(With help of generative AI)**

- **Learning Rate:** 
The learning rate varies between 1e-4 and 1e-2 with logarithmic sampling. The learning rate is crucial for convergence during training; too high can cause the model to oscillate or diverge, too low might lead to a long training process **(With help of generative AI)**

### 2.3 Error analysis

Evaluating best-performing system

1. Which system had a better accuracy on the test data?
2. Which system had a lower degree of overfitting?

1- The complex model achieved a test accuracy of **84.62%**, which is much higher than the simple model, which had a test accuracy of **61.54%**. This indicates that the complex model was able to learn more effectively from the training data and generalize better to the unseen test data.

2- To evaluate the degree of overfitting, we look at the gap between training accuracy (or loss) and validation accuracy (or loss) during the training process. A smaller gap indicates a lower degree of overfitting.

- **Complex Model:** The best validation accuracy was **80.84% (Epoch 10)**, and the training accuracy at that point was approximately **78.10%**. The gap between training and validation accuracy is relatively small, suggesting a lower degree of overfitting. Additionally, the early stopping callback terminated training at Epoch 13 due to a lack of improvement in validation loss, further helping to prevent overfitting.

- **Simple Model:** The best validation accuracy was **64.67% (Epoch 1)**, with the training accuracy around **42.64%**. The initial gap suggests a mismatch in performance, but this seems more like the model was not adequately learning rather than overfitting. Given the overall lower accuracy scores and the erratic behavior of the model's loss and accuracy, it's challenging to directly compare overfitting in the traditional sense. However, the simple model's training was stopped early at Epoch 4, indicating it wasn't able to improve significantly after the first epoch.

- **Conclusion:**
The complex model outperformed simple model on the test set but also showed signs of effective learning and generalization with a lower degree of overfitting compared to the simple model.






## 3 - A more complex classifier

### 3.1 Using ConvNets

Implement a model that uses a sequence of at least two `ConvD`, each one followed with `MaxPooling2D`. Use reasonable numbers for the hyperparameters (number of filters, kernel size, pool size, activation, etc), base on what we have seen in the lectures. Feel free to research the internet and / or generative AI to help you find a reasonable choice of hyperparameters. For this task, do not use pre-trained models.

#### 3.1.1 Compiling CNN model

First we find the best hyperparameters for the CNN model defined in the `build_model` function using the Keras Tuner library's Random Search algorithm. By tuning hyperparameters such as filter sizes, kernel sizes, pool sizes, dense layer units, and optimizer choice

In [19]:
def build_model(hp):
    Convmodel = Sequential([
        #2D convolution layer as the first layer, with tunable number of filters
        Conv2D(filters=hp.Int('filters_1', min_value=32, max_value=128, step=32),
               # The kernel sizecan be either 3x3 or 5x5, decided by the tuner (Use of Generative AI)
               kernel_size=hp.Choice('kernel_size_1', values=[3, 5]),
               activation='relu', input_shape=(230, 230, 3)),
        
        # Pool size that can either be 2x2 or 3x3 (Use of Generative AI)
        MaxPooling2D(pool_size=hp.Choice('pool_size_1', values=[2, 3])),
        
        Conv2D(filters=hp.Int('filters_2', min_value=64, max_value=256, step=32),
               kernel_size=hp.Choice('kernel_size_2', values=[3, 5]),
               activation='relu'),
        MaxPooling2D(pool_size=hp.Choice('pool_size_2', values=[2, 3])),
        
        #Flattens the input to transition to dense layers
        Flatten(),
        Dense(hp.Int('dense_units', min_value=128, max_value=512, step=128), activation='relu'),
        Dense(4, activation='softmax')
    ])

    Convmodel.compile(optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return Convmodel

early_stopping_conv = EarlyStopping(
    monitor='val_loss', # Monitor validation loss
    patience=3,         # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True # Restores model weights from the epoch with the best value of the monitored metric
)

# Initialize the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='my_dir',
    project_name='hparam_tuning'
)

tuner.search(
    training_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=[early_stopping_conv]
)

# Retrieve the best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters()[0]


print("Best Hyperparameters:")
for hyperparam, value in best_hyperparameters.values.items():
    print(f"{hyperparam}: {value}")

Trial 5 Complete [00h 12m 28s]
val_accuracy: 0.901197612285614

Best val_accuracy So Far: 0.901197612285614
Total elapsed time: 00h 45m 35s
Best Hyperparameters:
filters_1: 128
kernel_size_1: 3
pool_size_1: 3
filters_2: 128
kernel_size_2: 5
pool_size_2: 2
dense_units: 256
optimizer: adam


#### 3.1.2 Training Our CNN Model

In [21]:
history_cnn = best_model.fit(
    training_dataset,
    epochs=100,  # Higher epoch limit; early stopping will prevent reaching this if not necessary
    validation_data=validation_dataset,
    callbacks=[early_stopping_conv]
)

Epoch 1/100
25/25 [==============================] - 39s 2s/step - loss: 0.1147 - accuracy: 0.9654 - val_loss: 1.0516 - val_accuracy: 0.7964
Epoch 2/100
25/25 [==============================] - 39s 2s/step - loss: 0.1495 - accuracy: 0.9462 - val_loss: 0.8195 - val_accuracy: 0.8563
Epoch 3/100
25/25 [==============================] - 39s 2s/step - loss: 0.1716 - accuracy: 0.9449 - val_loss: 0.8479 - val_accuracy: 0.8263
Epoch 4/100
25/25 [==============================] - 43s 2s/step - loss: 0.1095 - accuracy: 0.9590 - val_loss: 1.3887 - val_accuracy: 0.7665
Epoch 5/100
25/25 [==============================] - 44s 2s/step - loss: 0.1051 - accuracy: 0.9526 - val_loss: 0.7719 - val_accuracy: 0.8743
Epoch 6/100
25/25 [==============================] - 44s 2s/step - loss: 0.1102 - accuracy: 0.9565 - val_loss: 0.6297 - val_accuracy: 0.9042
Epoch 7/100
25/25 [==============================] - 44s 2s/step - loss: 0.0390 - accuracy: 0.9898 - val_loss: 0.7534 - val_accuracy: 0.8862
Epoch 8/100
2

#### 3.1.3 CNN Model Results

In [22]:
# Evaluating the model on the test dataset
test_loss_conv, test_accuracy_conv = best_model.evaluate(test_dataset)

print(f"Test Loss: {test_loss_conv}")
print(f'Test accuracy: {test_accuracy_conv*100:.2f}%')

6/6 [==============================] - 3s 445ms/step - loss: 0.2420 - accuracy: 0.9112
Test Loss: 0.24201205372810364
Test accuracy: 91.12%


### 3.2 Using pre-trained models


#### 3.2.1 MobileNet Model Compiling

In [23]:
# Loading MobileNetV2 with pre-trained ImageNet weights, excluding the top (classification) layer
base_model = tf.keras.applications.MobileNetV2(input_shape=(230, 230, 3),
                                               include_top=False,
                                               weights='imagenet')

# Freeze the layers of the base_model
base_model.trainable = False

# Create the custom head for our dataset (replacing the top layer of MobileNet)
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Add a global spatial average pooling layer
x = Dense(1024, activation='relu')(x)  # Add a fully-connected layer
predictions = Dense(4, activation='softmax')(x)  # Add the final classification layer

# Construct the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Learning rate schedule (Use of Generative AI)
initial_learning_rate = 0.0001
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True)


# Use the legacy version of the Adam optimizer for better performance on M1/M2 Macs (Use of Generative AI)
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr_schedule)

# Compile the model with the modified optimizer
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 230, 230, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 115, 115, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 115, 115, 32  128         ['Conv1[0][0]']                  
                                )                                                             

 ization)                                                                                         
                                                                                                  
 block_3_expand_relu (ReLU)     (None, 58, 58, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 59, 59, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 29, 29, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 29, 29, 144)  576        ['block_3_depthwise[0][0]']      
 malizatio

                                                                                                  
 block_6_project_BN (BatchNorma  (None, 15, 15, 64)  256         ['block_6_project[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 15, 15, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 15, 15, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 15, 15, 384)  0           ['block_7_expand_BN[0][0]']      
          

 block_10_depthwise_BN (BatchNo  (None, 15, 15, 384)  1536       ['block_10_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 15, 15, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 15, 15, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 15, 15, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_

 block_14_expand_relu (ReLU)    (None, 8, 8, 960)    0           ['block_14_expand_BN[0][0]']     
                                                                                                  
 block_14_depthwise (DepthwiseC  (None, 8, 8, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 8, 8, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 8, 8, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_

#### 3.2.1 MobileNet Model Training

In [24]:
history_mobilenet = model.fit(
    training_dataset,
    epochs=10,
    validation_data=validation_dataset
)

Epoch 1/10
25/25 [==============================] - 7s 269ms/step - loss: 0.8008 - accuracy: 0.7145 - val_loss: 0.4453 - val_accuracy: 0.9102
Epoch 2/10
25/25 [==============================] - 6s 249ms/step - loss: 0.2926 - accuracy: 0.9475 - val_loss: 0.3056 - val_accuracy: 0.9162
Epoch 3/10
25/25 [==============================] - 6s 257ms/step - loss: 0.1868 - accuracy: 0.9706 - val_loss: 0.2547 - val_accuracy: 0.9341
Epoch 4/10
25/25 [==============================] - 6s 255ms/step - loss: 0.1377 - accuracy: 0.9808 - val_loss: 0.2269 - val_accuracy: 0.9341
Epoch 5/10
25/25 [==============================] - 6s 260ms/step - loss: 0.1080 - accuracy: 0.9821 - val_loss: 0.2079 - val_accuracy: 0.9341
Epoch 6/10
25/25 [==============================] - 6s 253ms/step - loss: 0.0874 - accuracy: 0.9846 - val_loss: 0.1953 - val_accuracy: 0.9341
Epoch 7/10
25/25 [==============================] - 6s 248ms/step - loss: 0.0723 - accuracy: 0.9872 - val_loss: 0.1866 - val_accuracy: 0.9401
Epoch 

#### 3.2.2 MobileNet Results

In [25]:
# Evaluating the model on the test dataset
test_loss_mobilenet, test_accuracy_mobilenet = model.evaluate(test_dataset)

print(f"Test Loss: {test_loss_mobilenet}")
print(f'Test accuracy: {test_accuracy_mobilenet*100:.2f}%')

6/6 [==============================] - 1s 173ms/step - loss: 0.1081 - accuracy: 0.9645
Test Loss: 0.10806741565465927
Test accuracy: 96.45%


### Task 3.3 Comparative evaluation

Analyzing and comparing the evaluation results of the top-performing systems

- Among all the systems, which one performed the best on the test set?
- Which type of weather was the most challenging to detect based on these accuracy scores.

The Mobilenet system performs better on the test set, achieving a test accuracy of **96.45%**, compared to the ConvModel system, which has a test accuracy of **91.12%**.

#### 3.3.1 Accuracy of your best system on each of the different weather categories:

In [29]:
predictions = model.predict(test_dataset)
predicted_classes = np.argmax(predictions, axis=1)

true_labels = np.concatenate([y for x, y in test_dataset], axis=0)

# confusion matrix
conf_matrix = tf.math.confusion_matrix(true_labels, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix.numpy())

# accuracy for each class
class_accuracies = np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)
for i, acc in enumerate(class_accuracies):
    print(f"Accuracy for class {i} (Weather Condition): {acc*100:.2f}%")

most_difficult_weather = np.argmin(class_accuracies)
print(f"The most difficult to detect weather condition is class {most_difficult_weather}.")

6/6 [==============================] - 1s 166ms/step
Confusion Matrix:
[[49  0  0  2]
 [ 0 34  0  0]
 [ 0  0 48  1]
 [ 2  0  1 32]]
Accuracy for class 0 (Weather Condition): 96.08%
Accuracy for class 1 (Weather Condition): 100.00%
Accuracy for class 2 (Weather Condition): 97.96%
Accuracy for class 3 (Weather Condition): 91.43%
The most difficult to detect weather condition is class 3.
